<a href="https://colab.research.google.com/github/michaelwnau/consequential-products/blob/main/URL_context_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Gemini API URL Context Tutorial

This Colab notebook demonstrates various use cases for the URL Context tool in Google's Gemini API.
The URL Context tool allows you to provide web URLs as context to AI models, enabling
analysis of web content without manual copying and pasting.

## Prerequisites
- Google AI API key
    - Generate a fresh API key in [Google AI Studio](https://aistudio.google.com/apikey)
    - Store the key as `GOOGLE_API_KEY` in the Colab Secrets manager 🔑
- Access to Gemini 2.5 models
- Python environment with required packages


### Setup and Installation


In [ ]:
import os
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch, UrlContext
from dotenv import load_dotenv
import json
from typing import List, Dict, Any

In [ ]:
# Load environment variables
load_dotenv()

# Initialize the client
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# genai.configure(api_key=GOOGLE_API_KEY) # Remove this line
client = genai.Client(api_key=GOOGLE_API_KEY) # Pass the API key here
model_id = "gemini-2.5-flash"

## Basic URL Context Usage
Let's start with a simple example comparing two recipes from different websites.

In [ ]:
def analyze_urls_basic(urls: List[str], prompt: str) -> Dict[str, Any]:
    """
    Basic function to analyze content from URLs using Gemini API

    Args:
        urls: List of URLs to analyze
        prompt: Analysis prompt that references the URLs

    Returns:
        Dictionary containing response text and metadata
    """
    tools = [{"url_context": {}}]

    # Create prompt with URLs
    full_prompt = f"{prompt}\n\nURLs to analyze: {', '.join(urls)}"

    response = client.models.generate_content(
        model=model_id,
        contents=full_prompt,
        config=GenerateContentConfig(tools=tools)
    )

    # Extract response text
    response_text = ""
    for part in response.candidates[0].content.parts:
        response_text += part.text

    # Get metadata about URL retrieval
    url_metadata = response.candidates[0].url_context_metadata

    return {
        "response": response_text,
        "url_metadata": url_metadata,
        "usage_metadata": response.usage_metadata
    }

This example is taken from [Google's URL Context documentation](https://ai.google.dev/gemini-api/docs/url-context#python)

In [ ]:
# Example 1: Recipe Comparison
recipe_urls = [
    "https://jessicainthekitchen.com/teriyaki-tofu-recipe/",
    "https://www.allrecipes.com/recipe/281059/crispy-teriyaki-tofu/"
]

recipe_prompt = "Compare the ingredients and cooking times from the recipes at these URLs. Highlight the key differences and similarities."

recipe_result = analyze_urls_basic(recipe_urls, recipe_prompt)
print("Recipe Comparison Results:")
print("=" * 50)
print(recipe_result["response"])
print("\nURL Retrieval Status:")
for url_meta in recipe_result["url_metadata"].url_metadata:
    print(f"URL: {url_meta.retrieved_url}")
    print(f"Status: {url_meta.url_retrieval_status}")
    print()

After running the cell you will have some results like:
<details>

>

Recipe Comparison Results:
==================================================
Both teriyaki tofu recipes aim for crispy tofu and a flavorful teriyaki sauce, but they differ in ingredients and cooking times.

### Key Similarities:

*   **Tofu Type:** Both recipes recommend using extra-firm tofu and pressing it to remove excess water.
*   **Crisping Agent:** Both use cornstarch to achieve a crispy texture for the tofu.
*   **Core Teriyaki Flavors:** Both recipes include soy sauce (or liquid aminos), ginger, garlic, and a sweetener in their teriyaki sauce.
*   **Garnish:** Both suggest using chopped scallions and sesame seeds as a garnish.
*   **Serving Suggestion:** Both recommend serving over rice.

### Key Differences:

#### Ingredients:

*   **Teriyaki Sauce Base:**
    *   Jessica in the Kitchen's recipe creates the teriyaki sauce from scratch using soy sauce, raw unrefined brown sugar (or coconut sugar), cornstarch, ground ginger, garlic powder, sesame seeds, rice wine vinegar, and water (or pineapple juice).
    *   Allrecipes' recipe uses pre-made vegetarian teriyaki sauce as its base, then enhances it with green onions, sherry, minced garlic, Sriracha sauce (optional), and ginger paste. This implies a simpler sauce preparation in the Allrecipes version.
*   **Sweetener:** Jessica in the Kitchen specifies raw unrefined brown sugar or coconut sugar. Allrecipes relies on the sweetness from the pre-made teriyaki sauce.
*   **Acidity:** Jessica in the Kitchen uses rice wine vinegar. Allrecipes uses sherry.
*   **Spice:** Allrecipes includes optional Sriracha sauce for a kick. Jessica in the Kitchen uses crushed red peppers for the teriyaki tofu marinade.
*   **Oil for Tofu:** Jessica in the Kitchen uses sesame oil (or regular oil) for cooking the tofu. Allrecipes specifies olive oil.
*   **Marinade for Tofu (before cooking):** Jessica in the Kitchen marinates the pressed tofu with light soy sauce (or liquid aminos), garlic powder, sea salt, black pepper, and cornstarch before cooking. Allrecipes only coats the pressed tofu in cornstarch before cooking.

#### Cooking Times:

*   **Prep Time:**
    *   Jessica in the Kitchen: 35 minutes (includes tofu pressing time).
    *   Allrecipes: 10 minutes.
*   **Cook Time:**
    *   Jessica in the Kitchen: 15 minutes (8 minutes to crisp tofu, 4 minutes to thicken sauce, 2-3 minutes for tofu to sit in sauce).
    *   Allrecipes: 15 minutes (5 minutes per side for tofu, 2 minutes to thicken sauce).
*   **Total Time:**
    *   Jessica in the Kitchen: 50 minutes.
    *   Allrecipes: 25 minutes.

In summary, the Allrecipes version is a quicker recipe with a shorter prep time, largely due to using a pre-made teriyaki sauce and a simpler tofu preparation before cooking. The Jessica in the Kitchen recipe involves making the teriyaki sauce from scratch and a more extensive tofu marinade, contributing to a longer prep and total time, but potentially offering more control over the flavor profile.

URL Retrieval Status:
URL: https://jessicainthekitchen.com/teriyaki-tofu-recipe/
Status: UrlRetrievalStatus.URL_RETRIEVAL_STATUS_SUCCESS

URL: https://www.allrecipes.com/recipe/281059/crispy-teriyaki-tofu/
Status: UrlRetrievalStatus.URL_RETRIEVAL_STATUS_SUCCESS
<details>




That useful! We might add a statement to save those results to a Google Drive "Recipes` folder and apply tags to the recipes, too.
What other use cases could we use this for?
* Content Analysis & Research
* Business Intelligence
* Content Creation & Synthesis
* Technical Documentation
* E-commerce & Product Research
* Legal & Compliance
* Education & Training
* Real Estate & Travel

